# Cypher queries for Neo4j

## Relationships

**created post data loading via ETL**

* could be used in demo - delete queries included
  
* (student)-[REGISTERED_ON]->(programme)
* (student)-[ENROLLED_ON]->(module)
* (activity)-[HAS_TYPE]->(activity_type)
* (module)-[HAS_OWNING_DEPT]->(department)
* (programme)-[HAS_OWNING_DEPT]->(department)

### (student)-[REGISTERED_ON]->(programme)



### (student)-[ENROLLED_ON]->(module)

// Create ENROLLED_ON relationship between students and modules 

// Match student, activity, and module nodes based on ATTEND and BELONGS_TO relationships
MATCH (s:student)-[:ATTENDS]->(a:activity)-[:BELONGS_TO]->(m:module)

// Create ENROLLED_ON relationship
MERGE (s)-[:ENROLLED_ON]->(m)

### (activity)-[HAS_TYPE]->(activity_type)

### (module)-[HAS_OWNING_DEPT]->(department)

### (programme)-[HAS_OWNING_DEPT]->(department)

## Database queries

### nodes without relationships

### property datatypes

In [ ]:
/* return datatype of actStartTime on activity node */

MATCH (a:activity)
RETURN DISTINCT apoc.meta.cypher.type(a.actStartTime) as actStartTimeType

### unique properties

In [ ]:
// List unique properties for a Node

MATCH (a:activity)
UNWIND keys(a) AS propertyKey
RETURN COLLECT(DISTINCT propertyKey) AS propertyKeys
//RETURN DISTINCT propertyKey as propertyKeys

### delete relationships

In [ ]:
// Delete all relationships

MATCH ()-[r]->()
DELETE r

### students witout activities

In [ ]:
// Students without Activities
MATCH (s:student)
WHERE NOT (s)-[:ATTENDS]->()
RETURN s

### activities without rooms

In [ ]:
// Activities without Rooms

MATCH (at:activityType)
WHERE NOT (at)<-[:HAS_TYPE]-()
RETURN at;

## Indexes

* **Frequently Queried Properties**: Properties that you often use in WHERE clauses or for lookups.
* **Properties Used in MATCH Clauses**: Properties involved in matching nodes for relationship creation or traversal.
* **High-Cardinality Properties**: Properties with many distinct values benefit from indexing, as they help narrow down search results quickly.

In [ ]:
// Show indexes
SHOW INDEXES;

### Create Indexes

## Count queries

### Node count - by label

In [ ]:
// Count of nodes - row per node

UNWIND ["student", "staff", "room", "activity"] AS label
MATCH (n)
WHERE label IN labels(n)
RETURN label, count(n) AS count

In [ ]:
// Count of nodes - single row

MATCH (n:student)
WITH count(n) AS studentCount
MATCH (n:staff)
WITH studentCount, count(n) AS lecturerCount
MATCH (n:room)
WITH studentCount, lecturerCount, count(n) AS roomCount
MATCH (n:activity)
RETURN studentCount, lecturerCount, roomCount, count(n) AS activityCount

### Relationship count - by type

In [ ]:
// Count of relationships

MATCH ()-[r:ATTENDS]->()
WITH count(r) AS attendsCount
MATCH ()-[r:TEACHES]->()
WITH attendsCount, count(r) AS teachesCount
MATCH ()-[r:OCCUPIES]->()
RETURN attendsCount, teachesCount, count(r) AS occupiesCount

### Activity

### Activities on a day

In [ ]:
// Count of activities on a day

MATCH (a:activity)
WHERE a.actDayName = "Wednesday"
RETURN DISTINCT count(a) AS wednesdayActivities

### Activities per day

In [ ]:
// Count of activities per day

MATCH (a:activity)
RETURN DISTINCT a.actDayName AS dayName, count(a) AS activityCount

### Staff activity

In [ ]:
// Staff activity count

MATCH (st:staff)-[r:TEACHES]->(a:activity)
RETURN st.staffFullName_anon AS staffName, count(a) AS activityCount
ORDER BY activityCount DESC

### Activity for a start time

In [ ]:
// Activity count by time (start)

MATCH (a:activity)
WHERE a.actStartTime = localtime("17:00:00")
//AND a.actDayName = "Wednesday"
RETURN count(a) AS activitiesStartingAt5pm

## Hard Constraints

Hard constraints are generally rules or conditions which cannot be violated.  Violation would indicate non-viable timetable, e.g. a lecturer being scheduled to teach in two places simultaneously.

In reality, hard constraints appear in timetables and are accepted with real-world workarounds.

* **All Activities Scheduled**: Every lecture, tutorial, lab, etc., must have a designated time and place.
* **No Room Conflicts (aka room clash)**: Two activities cannot be scheduled in the same room at the same time.
* **No Staff Conflicts (aka staff clash)**: A staff member cannot be assigned to two activities or more occurring at the same datetime.
* **No Student Conflicts (aka student clash)**: A student cannot be allocated to two or more activities which are scheduled at the same datetime.
* **Staff Availability Respected**: Activities cannot be scheduled during a staff member's unavailable times (e.g., research days, meetings, unavailability pattern).
* **Room Capacity Sufficient**: The room assigned to an activity must accommodate the expected number of students
* **Curriculum Requirements Met**: Required courses must be offered at times when students can take them

### Unscheduled activities

* No start time, start date, end date
* Could be changed to use 'isSheduled' property

In [ ]:
/**
 * This Cypher query retrieves all activities that have missing start date, start time, or end time.
 * It matches nodes with the label "activity" and checks if the properties "actStartDate", "actStartTime", or "actEndTime" are null.
 * If any of these properties is null, the node is returned.
 * 
 * @returns {Node} The activities with missing start date, start time, or end time.




MATCH (a:activity)
WHERE a.actStartDate IS NULL 
OR a.actStartTime IS NULL 
OR a.actEndTime IS NULL
RETURN a

### Room clashes

In [ ]:
/**
 * This Cypher query matches activities (a1 and a2) located in the same room (r) and occurring on the same date.
 * It filters out activities that are the same (a1 <> a2) and checks for overlapping time intervals.
 * The query returns the matched activities (a1 and a2) along with the room (r) they are located in.

 * Needs to be tweaked (or data changed) to handle jointly-taught or variant activities.  Use case for graph structure TBC.
 */
 
MATCH (a1:activity)-[r1:OCCUPIES]->(r:room)<-[r2:OCCUPIES]-(a2:activity)
WHERE a1.actStartDate = a2.actStartDate AND a1 <> a2
    AND (
        (a1.actStartTime <= a2.actStartTime AND a1.actEndTime > a2.actStartTime)
        OR 
        (a2.actStartTime <= a1.actStartTime AND a2.actEndTime > a1.actStartTime)
    )
RETURN a1, a2, r, r1, r2

#### Room clash example - room with known clashes

In [ ]:
// Find clashes (overlapping activities) within specific rooms - EXAMPLE

MATCH (a1:activity)-[r1:OCCUPIES]->(r:room)<-[r2:OCCUPIES]-(a2:activity)
WHERE r.roomName IN ["4Q50/51 FR", "4Q69 FR", "3E Maths Open Zone A", "3E12 FR"] 
  AND a1.actStartDate = a2.actStartDate 
  AND a1 <> a2
  AND (
        (a1.actStartTime <= a2.actStartTime AND a1.actEndTime > a2.actStartTime)
        OR 
        (a2.actStartTime <= a1.actStartTime AND a2.actEndTime > a1.actStartTime)
      )
RETURN a1, a2, r, r1, r2

#### Room clash with demo data

In [ ]:
// to prove room clash query - load below
// 2x rooms, 5x activities, 2x clashes

CREATE 
  (room1:Room {name: "Room A"}),
  (room2:Room {name: "Room B"}),
  (a1:Activity {name: "Graph Lecture", Date: date("2024-06-17"), StartTime: "10:00", EndTime: "11:30"}),
  (a2:Activity {name: "Networks Lab", Date: date("2024-06-17"), StartTime: "11:00", EndTime: "13:00"}),
  (a3:Activity {name: "Data Science Tutorial", Date: date("2024-06-18"), StartTime: "09:00", EndTime: "10:30"}),
  (a4:Activity {name: "Graph Seminar", Date: date("2024-06-18"), StartTime: "10:00", EndTime: "12:00"}),
  (a5:Activity {name: "Computer Science Workshop", Date: date("2024-06-17"), StartTime: "14:00", EndTime: "16:00"}),
  (a1)-[:LOCATED_IN]->(room1),
  (a2)-[:LOCATED_IN]->(room1), 
  (a3)-[:LOCATED_IN]->(room2),
  (a4)-[:LOCATED_IN]->(room2),
  (a5)-[:LOCATED_IN]->(room1) 


In [ ]:

// delete activities

MATCH (a:Activity)
WHERE a.name IN ["Graph Lecture", "Networks Lab", "Data Science Tutorial", "Graph Seminar", "Computer Science Workshop"]
DETACH DELETE a


In [ ]:

// delete rooms

MATCH (r:Room)
WHERE r.name IN ["Room A", "Room B"]
DELETE r

### Room Capacity violation

In [ ]:
// Room capacity violation

/**
 * This Cypher query retrieves rooms, activities, and the number of students attending each activity.
 * It then filters the results to only include rooms that have more students attending than their capacity.
 * The query returns the room, activity date, activity name, room capacity, and the number of extra students needed to fill the room.
 *
 * @returns {Object[]} An array of objects containing the following properties:
 *   - r: The room node
 *   - a.Date: The date of the activity
 *   - Activity: The name of the activity
 *   - roomCapacity: The capacity of the room
 *   - extraNeeded: The number of extra students needed to fill the room
 */

MATCH (r:room)<-[r1:OCCUPIES]-(a:activity)<-[:ATTENDS]-(s:student)
//WHERE a.Date >= date("2022-01-01") AND a.Date <= date("2022-06-30") 
WITH r, a, count(s) as numStudents
WHERE numStudents > r.roomCapacity
RETURN r, a.actStartDate, a.actName AS Activity, r.roomCapacity, numStudents - r.roomCapacity AS extraNeeded
ORDER BY extraNeeded DESC

### student clashes





In [ ]:
MATCH (s:student)-[:ATTENDS]->(a1:activity)
WITH s, a1
MATCH (s)-[:ATTENDS]->(a2:activity) 
WHERE a1 <> a2 
  AND a1.actStartDate = a2.actStartDate 
  AND (a1.actStartTime < a2.actEndTime AND a1.actEndTime > a2.actStartTime)  // Correct overlap condition
  AND NOT (a1.actStartTime = a2.actEndTime OR a1.actEndTime = a2.actStartTime) // Exclude "touching" cases
RETURN s.stuFirstName_anon AS Student, 
       a1.actStartDate AS ClashDate, 
       a1.actName AS Activity1, 
       a1.actStartTime + "-" + a1.actEndTime AS Timeslot1, 
       a2.actName AS Activity2, 
       a2.actStartTime + "-" + a2.actEndTime AS Timeslot2
ORDER BY Student, ClashDate;



## Soft constraints

Soft constraints can be considered to be (strong) preferences.  They should be generally met and only violated when absolutely necessary.  For example, a member of staff may be unavailable on Fridays, generally, but they are scheduled to teach on one Friday by prior arrangement.  Other examples might include ensuring that students have an opportunity to eat lunch or minimising travel between distant locations. 

* **Minimal Idle Time (aka no large gaps):** Minimise gaps in staff and student schedules (within reason).
* **Spread Activities (aka maximum consecutive hours):** Avoid clumping all activities for a student or staff member on one day.
* **Preferred Times:** Consider staff and student preferences for morning, afternoon, or evening classes
* **Travel Time:** Minimise the time students need to travel between consecutive classes (especially on large campuses), e.g. between building blocks or by lat/long
* **Lunch Breaks:** Ensure students have sufficient time for lunch breaks.
* **Consistent Days/Times:** Try to schedule recurring activities (e.g., weekly lectures) on the same day and time.

In [ ]:
// students with gaps between activities

MATCH (s:student)-[:ATTENDS]->(a:activity)
WITH s, a
ORDER BY s.stuFirstName_anon, a.actStartDate, a.actStartTime

// Group activities by student and date
WITH s, a.actStartDate AS date, collect({start: a.actStartTime, end: a.actEndTime, activity: a}) AS times

// Calculating the gaps in hours between consecutive activities
WITH s, date, times, 
     [i IN range(0, size(times)-2) | 
      {gap: duration.between(times[i].end, times[i+1].start).minutes / 60.0, 
       firstActivity: times[i].activity, 
       secondActivity: times[i+1].activity}] AS gaps

// Filtering gaps based on a threshold of 3 hours
WITH s, date, gaps
WHERE any(gapRecord IN gaps WHERE gapRecord.gap > 6.0)

// Finding the maximum gap that exceeds the threshold
WITH s, date, reduce(maxGap = {gap: 0.0, firstActivity: null, secondActivity: null}, gapRecord IN gaps | 
    CASE WHEN gapRecord.gap > maxGap.gap THEN gapRecord ELSE maxGap END) AS maxGapRecord

// Returning the result
RETURN s.stuID_anon AS student, 
       date AS activityDate,
       maxGapRecord.firstActivity AS activity1, 
       maxGapRecord.secondActivity AS activity2,
       maxGapRecord.gap AS maxGapInHours
ORDER BY s.stuFirstName_anon


In [ ]:
MATCH (s:student)-[:ATTENDS]->(a:activity)
WHERE s.stuID_anon = "stu-10085720"
AND a.actStartDate = date("2022-10-03")
WITH s, a
ORDER BY a.actStartTime

// Collecting the start and end times of the activities
WITH s, collect({start: a.actStartTime, end: a.actEndTime}) AS times

// Calculating the gaps in minutes between consecutive activities
WITH s, times, 
     [i IN range(0, size(times)-2) | 
      duration.between(times[i].end, times[i+1].start).minutes / 60.0] AS gaps

// Finding the maximum gap
RETURN s.stuID_anon AS student, times, gaps, reduce(maxGap = 0.0, gap IN gaps | CASE WHEN gap > maxGap THEN gap ELSE maxGap END) AS maxGap

How this could be used - pre-calculate and store as a property on the student, or as a separate node.

In [ ]:
// Calculates - total hours, max block hours, max block activities per day 
// to be used for max block hours and max block activities per day
// logic - example

MATCH (s:student {stuID_anon:"stu-10085720"})-[:ATTENDS]->(a:activity)
WITH s, a ORDER BY a.actStartDate, a.actStartTime
WITH s, a.actStartDate AS Date, 
     SUM(a.actDurationInMinutes) / 60.0 AS totalHours,
     REDUCE(
        blockInfo = [],
        activity IN COLLECT(a)
        | CASE
            WHEN blockInfo = [] THEN [[activity]]
            ELSE CASE
                   WHEN head(last(blockInfo)).actEndTime >= activity.actStartTime
                     THEN blockInfo[..-1] + [last(blockInfo) + activity]
                   ELSE blockInfo + [[activity]]
                 END
          END
     ) AS blocks
UNWIND blocks AS block
WITH s, Date, totalHours, blocks,
     REDUCE(blockHours = 0.0, activity IN block | blockHours + activity.actDurationInMinutes) / 60.0 AS blockHours,
     SIZE(block) AS blockActivities
RETURN s.stuFullName_anon AS Student, Date, totalHours, 
       MAX(blockHours) AS blockHours,
       MAX(blockActivities) AS blockActivities
ORDER BY Date;

Explanation:

1. Match and Sort Activities:

* MATCH (s:student {stuID_anon:"stu-10085720"})-[:ATTENDS]->(a:activity): Matches the specified student and all their attended activities.
* WITH s, a ORDER BY a.actStartDate, a.actStartTime: Sorts the activities by their start date and then by their start time within each date.

2. Calculate Total Hours and Group Activities into Blocks:

* WITH s, a.actStartDate AS Date, SUM(a.actDurationInMinutes) / 60.0 AS totalHours, ...: Calculates the total hours spent on activities for each date by summing the durations of all activities on that date and converting minutes to hours.
* REDUCE(blockInfo = [], activity IN COLLECT(a) | ...): Groups activities into blocks based on time overlaps using a REDUCE function and a CASE expression.
  * It initialises an empty list blockInfo to store the blocks.
  * It iterates over the collected activities (COLLECT(a)).
  * For each activity:
    * If blockInfo is empty (first activity), it creates a new block with the activity.
    * Otherwise, it checks if the current activity overlaps with the last activity in the last block of blockInfo.
      * If there's an overlap, it adds the current activity to the last block.
      * If there's no overlap, it creates a new block with the current activity.

3. Calculate Block Hours and Number of Activities:

* UNWIND blocks AS block: Unwinds the list of blocks, processing each block individually.
* WITH s, Date, totalHours, blocks, REDUCE(blockHours = 0.0, activity IN block | blockHours + activity.actDurationInMinutes) / 60.0 AS blockHours, SIZE(block) AS blockActivities: For each block, it calculates the total duration in hours (blockHours) by summing the durations of activities within the block and converting minutes to hours. It also calculates the number of activities in the block (blockActivities).

4. Return Aggregated Results:

* RETURN s.stuFullName_anon AS Student, Date, totalHours, MAX(blockHours) AS blockHours, MAX(blockActivities) AS blockActivities ORDER BY Date;: Returns the student's full name, the date, the total hours for the day, the maximum block hours across all blocks for that day, and the maximum number of activities within a single block for that day. The results are ordered by date.

In [ ]:
MATCH (s:student)-[:ATTENDS]->(a:activity)<-[:TEACHES]-(:staff) // Filter for teaching activities
WITH s, a ORDER BY a.actStartDate, a.actStartTime
WITH s, a.actStartDate AS Date, 
     SUM(a.actDurationInMinutes) / 60.0 AS totalHours,
     REDUCE(
         blockInfo = [],
         activity IN COLLECT(a)
         | CASE
             WHEN blockInfo = [] THEN [[activity]]
             ELSE CASE
                     WHEN head(last(blockInfo)).actEndTime >= activity.actStartTime
                         THEN blockInfo[..-1] + [last(blockInfo) + activity]
                     ELSE blockInfo + [[activity]]
                 END
         END
     ) AS blocks
UNWIND blocks AS block
WITH s, Date, totalHours, blocks,
     REDUCE(blockHours = 0.0, activity IN block | blockHours + activity.actDurationInMinutes) / 60.0 AS blockHours,
     SIZE(block) AS blockActivities
WHERE blockHours > 5 // Filter for blocks with more than 5 hours
RETURN s.stuFullName_anon AS Student, Date, totalHours, 
       blockHours,
       blockActivities
ORDER BY Date;


In [ ]:
how this could be used - precalculate and add to student-date relationship

### total hours per day 

In [ ]:
MATCH (s:student )-[:ATTENDS]->(a:activity)
WITH s, a.actStartDate AS Date, SUM(a.actDurationInMinutes) / 60.0 AS totalHours
RETURN s.stuFullName_anon AS Student, Date, totalHours
ORDER BY Date;

### longest consecutive block per day

In [ ]:
MATCH (s:student {stuFullName_anon: "Susan Lopez"})-[:ATTENDS]->(a:activity {actStartDate: date("2022-09-27")})
WITH s, a 
ORDER BY a.actStartTime
WITH s, COLLECT(a) AS activities
WITH s, activities,
     REDUCE(
       state = {currentBlock: {duration: 0, start: null, end: null}, longestBlock: {duration: 0, start: null, end: null}},
       activity IN activities |
         CASE
           WHEN state.currentBlock.end IS NULL OR 
                activity.actStartTime > state.currentBlock.end
           THEN {
             currentBlock: {
               duration: activity.actDurationInMinutes,
               start: activity.actStartTime,
               end: activity.actEndTime
             },
             longestBlock: 
               CASE
                 WHEN activity.actDurationInMinutes > state.longestBlock.duration
                 THEN {
                   duration: activity.actDurationInMinutes,
                   start: activity.actStartTime,
                   end: activity.actEndTime
                 }
                 ELSE state.longestBlock
               END
           }
           ELSE {
             currentBlock: {
               duration: (activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                         (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute),
               start: state.currentBlock.start,
               end: activity.actEndTime
             },
             longestBlock: 
               CASE
                 WHEN ((activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                       (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute)) > state.longestBlock.duration
                 THEN {
                   duration: (activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                             (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute),
                   start: state.currentBlock.start,
                   end: activity.actEndTime
                 }
                 ELSE state.longestBlock
               END
           }
         END
     ) AS finalState
RETURN
  s.stuFullName_anon AS stuName,
  activities[0].actStartDate AS date,
  finalState.longestBlock.duration AS longestConsecutiveBlockDuration,
  finalState.longestBlock.start AS blockStartTime,
  finalState.longestBlock.end AS blockEndTime

In [ ]:
// example student

MATCH (s:student {stuFullName_anon: "Susan Lopez"})-[:ATTENDS]->(a:activity {actStartDate: date("2022-09-27")})
WITH s, a 
ORDER BY a.actStartTime
WITH s, COLLECT(a) AS activities
WITH s, activities,
     REDUCE(
       state = {currentBlock: {duration: 0, start: null, end: null}, longestBlock: {duration: 0, start: null, end: null}},
       activity IN activities |
         CASE
           WHEN state.currentBlock.end IS NULL OR 
                activity.actStartTime > state.currentBlock.end
           THEN {
             currentBlock: {
               duration: activity.actDurationInMinutes,
               start: activity.actStartTime,
               end: activity.actEndTime
             },
             longestBlock: 
               CASE
                 WHEN activity.actDurationInMinutes > state.longestBlock.duration
                 THEN {
                   duration: activity.actDurationInMinutes,
                   start: activity.actStartTime,
                   end: activity.actEndTime
                 }
                 ELSE state.longestBlock
               END
           }
           ELSE {
             currentBlock: {
               duration: (activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                         (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute),
               start: state.currentBlock.start,
               end: activity.actEndTime
             },
             longestBlock: 
               CASE
                 WHEN ((activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                       (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute)) > state.longestBlock.duration
                 THEN {
                   duration: (activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                             (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute),
                   start: state.currentBlock.start,
                   end: activity.actEndTime
                 }
                 ELSE state.longestBlock
               END
           }
         END
     ) AS finalState
RETURN
  s.stuFullName_anon AS stuName,
  activities[0].actStartDate AS date,
  finalState.longestBlock.duration AS longestConsecutiveBlockDuration,
  finalState.longestBlock.start AS blockStartTime,
  finalState.longestBlock.end AS blockEndTime

### max hours per day

In [ ]:
// sum of activity durations
// does not account for simultaneous activities (clashes) - so could be inflated, e.g. 12.5 hour students

MATCH (s:student)-[:ATTENDS]->(a:activity)
WITH s, a.actStartDate AS Date, SUM(a.actDurationInMinutes) / 60.0 AS totalHours
WHERE totalHours > 7 // Set  maximum here
RETURN s.stuFullName_anon AS Student, Date, totalHours
ORDER BY Date;

### max consecutive hours per day



In [ ]:
// example student

MATCH (s:student {stuFullName_anon: "Susan Lopez"})-[:ATTENDS]->(a:activity {actStartDate: date("2022-09-27")})
WITH s, a 
ORDER BY a.actStartTime
WITH s, COLLECT(a) AS activities
WITH s, activities,
     REDUCE(
       state = {currentBlock: {duration: 0, start: null, end: null}, longestBlock: {duration: 0, start: null, end: null}},
       activity IN activities |
         CASE
           WHEN state.currentBlock.end IS NULL OR 
                activity.actStartTime > state.currentBlock.end
           THEN {
             currentBlock: {
               duration: activity.actDurationInMinutes,
               start: activity.actStartTime,
               end: activity.actEndTime
             },
             longestBlock: 
               CASE
                 WHEN activity.actDurationInMinutes > state.longestBlock.duration
                 THEN {
                   duration: activity.actDurationInMinutes,
                   start: activity.actStartTime,
                   end: activity.actEndTime
                 }
                 ELSE state.longestBlock
               END
           }
           ELSE {
             currentBlock: {
               duration: (activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                         (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute),
               start: state.currentBlock.start,
               end: activity.actEndTime
             },
             longestBlock: 
               CASE
                 WHEN ((activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                       (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute)) > state.longestBlock.duration
                 THEN {
                   duration: (activity.actEndTime.hour * 60 + activity.actEndTime.minute) - 
                             (state.currentBlock.start.hour * 60 + state.currentBlock.start.minute),
                   start: state.currentBlock.start,
                   end: activity.actEndTime
                 }
                 ELSE state.longestBlock
               END
           }
         END
     ) AS finalState
WHERE finalState.longestBlock.duration > 270 // Filter for blocks with more than 4.5 hours
RETURN
  s.stuFullName_anon AS stuName,
  activities[0].actStartDate AS date,
  finalState.longestBlock.duration AS longestConsecutiveBlockDuration,
  finalState.longestBlock.start AS blockStartTime,
  finalState.longestBlock.end AS blockEndTime

### Lunch break

In [ ]:
MATCH (s:student)-[:ATTENDS]->(a:activity)
WITH s, a
ORDER BY a.actStartDate, a.actStartTime
WITH s, COLLECT(a) AS activities
UNWIND activities AS activity
WITH s.stuFullName_anon AS Student, activity.actStartDate AS Date, time(activity.actStartTime) AS StartTime, time(activity.actEndTime) AS EndTime, duration.between(time('12:00'), time('14:00')).minutes AS BreakWindow_12_14
WITH Student, Date, BreakWindow_12_14, COLLECT([StartTime, EndTime]) AS Activities
UNWIND Activities AS activity
WITH Student, Date, BreakWindow_12_14, activity[0] AS StartTime, activity[1] AS EndTime
WITH Student, Date, BreakWindow_12_14,
     CASE
       WHEN StartTime >= time('14:00') OR EndTime <= time('12:00') THEN 0
       WHEN StartTime < time('12:00') AND EndTime > time('14:00') THEN BreakWindow_12_14
       WHEN StartTime >= time('12:00') AND StartTime < time('14:00') THEN duration.between(StartTime, time('14:00')).minutes
       WHEN EndTime > time('12:00') AND EndTime <= time('14:00') THEN duration.between(time('12:00'), EndTime).minutes
     END AS BookedDurationMinutes
RETURN Student, Date, BreakWindow_12_14, BreakWindow_12_14 - SUM(BookedDurationMinutes) AS FreeTimeMinutes, SUM(BookedDurationMinutes) AS BookedTimeMinutes
ORDER BY Date

In [ ]:
// example - 12:00-14:00

MATCH (s:student {stuFullName_anon: "Susan Lopez"})-[:ATTENDS]->(a:activity{actStartDate:date("2022-09-27")})
WITH s, a
ORDER BY a.actStartDate, a.actStartTime
WITH s, COLLECT(a) AS activities
UNWIND activities AS activity
WITH s.stuFullName_anon AS Student, activity.actStartDate AS Date, time(activity.actStartTime) AS StartTime, time(activity.actEndTime) AS EndTime, duration.between(time('12:00'), time('14:00')).minutes AS BreakWindow_12_14
WITH Student, Date, BreakWindow_12_14, COLLECT([StartTime, EndTime]) AS Activities
UNWIND Activities AS activity
WITH Student, Date, BreakWindow_12_14, activity[0] AS StartTime, activity[1] AS EndTime
WITH Student, Date, BreakWindow_12_14,
     CASE
       WHEN StartTime >= time('14:00') OR EndTime <= time('12:00') THEN 0
       WHEN StartTime < time('12:00') AND EndTime > time('14:00') THEN BreakWindow_12_14
       WHEN StartTime >= time('12:00') AND StartTime < time('14:00') THEN duration.between(StartTime, time('14:00')).minutes
       WHEN EndTime > time('12:00') AND EndTime <= time('14:00') THEN duration.between(time('12:00'), EndTime).minutes
     END AS BookedDurationMinutes
RETURN Student, Date, BreakWindow_12_14, BreakWindow_12_14 - SUM(BookedDurationMinutes) AS FreeTimeMinutes, SUM(BookedDurationMinutes) AS BookedTimeMinutes
ORDER BY Date

## rooms

In [ ]:
MATCH (r:room)
WHERE r.roomLatitude IS NOT NULL AND r.roomLongitude IS NOT NULL
SET r.location = point({latitude: toFloat(r.roomLatitude), longitude: toFloat(r.roomLongitude)})

In [ ]:
MATCH (room1:room), (room2:room)
WHERE room1 <> room2 
RETURN room1.roomName, room2.roomName, 
       point.distance(room1.location, room2.location) AS distanceInMeters

* walking speed - 1.4 m/s -> divide distnace by 1.4 to get time in seconds
  * does not consider actual walking routes

In [ ]:
MATCH (r:demoRoom)
SET r.location = point({latitude: toFloat(r.lat), longitude: toFloat(r.long)})

In [ ]:
// possible use:

MATCH (s:Student)-[:ATTENDS]->(a1:Activity)-[:LOCATED_IN]->(r1:demoRoom),
      (s)-[:ATTENDS]->(a2:Activity)-[:LOCATED_IN]->(r2:demoRoom)
WHERE a1 <> a2 AND 
      // Assuming activities have a 'startTime' property to determine adjacency
      a2.startTime > a1.startTime AND 
      duration.between(a1.startTime, a2.startTime) < duration({minutes: 10}) // Adjust as needed
RETURN s.name, a1.name, a2.name, 
       point.distance(r1.location, r2.location) AS distanceInMeters

In [ ]:
MATCH (s:student)-[:ATTENDS]->(a1:activity)-[:OCCUPIES]->(r1:room),
      (s)-[:ATTENDS]->(a2:activity)-[:OCCUPIES]->(r2:room)
WHERE a1 <> a2 AND 
      // Assuming activities have a 'startTime' property to determine adjacency
      a2.actStartTime > a1.actStartTime AND 
      duration.between(a1.actStartTime, a2.actStartTime) < duration({minutes: 10}) // Adjust as needed
RETURN s.name, a1.name, a2.name, 
       point.distance(r1.location, r2.location) AS distanceInMeters

1. Define a Penalty Function:

* Decide on a penalty amount (x) to deduct for each violation.
* Consider factors like:
  * The distance between rooms (higher distance, higher penalty).
  * The time between activities (shorter time, higher penalty). 
  * Any other relevant factors in your scenario.

2. Modify the Cypher Query:

* Add a CASE statement to calculate the penalty based on the distance and other factors.
* Aggregate the penalties for each student.

In [ ]:
MATCH (s:Student)-[:ATTENDS]->(a1:Activity)-[:OCCUPIES]->(r1:room),
      (s)-[:ATTENDS]->(a2:Activity)-[:OCCUPIES]->(r2:room)
WHERE a1 <> a2 AND 
      a2.startTime > a1.startTime AND 
      duration.between(a1.startTime, a2.startTime) < duration({minutes: 10})
WITH s, a1, a2, point.distance(r1.location, r2.location) AS distanceInMeters
// Calculate penalty based on distance and other factors (adjust as needed)
WITH s, SUM(
    CASE 
        WHEN distanceInMeters > 500 THEN 2*x // Higher penalty for longer distances
        WHEN distanceInMeters > 200 THEN x
        ELSE 0
    END
) AS totalPenalty
// Assuming students have a 'timetableQualityScore' property
SET s.timetableQualityScore = s.timetableQualityScore - totalPenalty
RETURN s.name, s.timetableQualityScore